In [22]:
import pandas as pd

In [39]:
import re

In [1]:
from pymongo import MongoClient
from bson import ObjectId

In [2]:
def mystring(item , depth = 0):
    out = ''
    mydepth = depth
    
    if type(item) == list:
        for it in item:
            out += '\n'
            newdepth = mydepth + 1
            out += mystring(it, newdepth)
    
    elif type(item) == dict:
        for key in item:
            out += '  ' * mydepth + key + ': '
            newdepth = mydepth + 1
            out += mystring(item[key], newdepth)

    else:
        out += '{0}'.format(item) + '\n'
    
    return out

In [3]:
client = MongoClient()

In [5]:
tflora = client.ThreatenedFlora
taxo = tflora.Taxonomy

In [94]:
qu = taxo.aggregate([
    {"$match": {"family": "Arecaceae"}},
    {"$lookup":
        {"localField": "_id",
        "from" : "Samples",
        "foreignField": "taxonID",
        "as": "specimen"}
    },
    {"$project": 
        {"_id": 1,
        "genus": 1,
        "specificEpithet": 1,
        "specimen.TaxonID": 1,
        "specimen.recordedBy": 1,
        "specimen.recordNumber": 1,
        "specimen.decimalLatitude": 1,
        "specimen.decimalLongitude": 1}
    }#,
    #{"$limit": 5}
])

dat2pan = {'Taxon': [], 'Latitude' : [], 'Longitude': []}

for spp in qu:
    species = spp['genus'] + " " + spp['specificEpithet']
    for sam in spp['specimen']:
        #print(species, sam['decimalLongitude'], sam['decimalLatitude'])
        dat2pan['Taxon'].append(species)
        dat2pan['Latitude'].append(sam['decimalLatitude'])
        dat2pan['Longitude'].append(sam['decimalLongitude'])
    #print(mystring(spp))    

colls = pd.DataFrame.from_dict(dat2pan)

In [100]:
colls = colls.drop(colls[colls.Latitude.isna() & colls.Longitude.isna()].index)

In [101]:
colls.to_csv('arecaceae.csv', index=False)

In [93]:
qu = taxo.aggregate([
    {"$match": {"family": "Arecaceae"}},
    {"$project": 
        {"_id": 1,
        "genus": 1,
        "specificEpithet": 1,
        'threatStatus': 1,
        'threatCriteria': 1}
    }#,
    #{"$limit": 5}
])

dat2pan = {'Taxon': [], 'Category' : [], 'Criteria': []}

for spp in qu:
    species = spp['genus'] + " " + spp['specificEpithet']
    dat2pan['Taxon'].append(species)
    dat2pan['Category'].append(spp['threatStatus'])
    dat2pan['Criteria'].append(spp['threatCriteria'])

    #print(species, spp['threatCriteria'])
    #print(species, spp['threatStatus'], spp['threatCriteria'])
    #print(mystring(spp))
    
pd.DataFrame.from_dict(dat2pan).to_csv('Arecaceae_categories.csv', index=False)

In [37]:
taxo.find_one()

{'_id': ObjectId('5eff663d7b5e9c69d6a8dd03'),
 'genus': 'Acidodontium',
 'specificEpithet': 'longifolium',
 'project': 'Vedas',
 'kingdom': 'Plantae',
 'phylum': 'Bryophyta',
 'family': 'Bryaceae',
 'scientificName': 'Acidodontium longifolium',
 'infraspecificEpithet': None,
 'taxonRank': 'Especie',
 'scientificNameAuthorship': '(Paris) Broth.',
 'threatStatus': 'LC',
 'threatCriteria': None,
 'threatStatusScope': 'National',
 'distribution': 'Nativa',
 'comments': None}

In [18]:
samples = tflora.Samples

In [19]:
samples.find_one()

{'_id': ObjectId('5eff66707b5e9c69d6a8e1bf'),
 'myindex': 0,
 'source': 'Bosque_seco',
 'occurrenceID': None,
 'basisOfRecord': 'Espécimen preservado',
 'institutionCode': 'UCALDAS',
 'collectionCode': 'FAUC',
 'catalogNumber': 'FAUC6108',
 'institutionID': None,
 'occurrenceRemarks': 'Arbusto de 3-4 m. Flores azules, fruto con semillas rojas. Cultivado como ornamental en el CIAT',
 'recordNumber': None,
 'recordedBy': 'M. De Fraume',
 'reproductiveCondition': 'Floración y Fructificación',
 'occurrenceStatus': None,
 'disposition': None,
 'otherCatalogNumbers': None,
 'eventDate': '1990-06-06',
 'eventRemarks': None,
 'habitat': None,
 'continent': 'SA',
 'country': 'Colombia',
 'countryCode': 'CO',
 'stateProvince': 'Valle del Cauca',
 'county': 'Palmira',
 'municipality': 'Palmira',
 'locality': 'CIAT',
 'verbatimLocality': None,
 'minimumElevationInMeters': 980.0,
 'maximumElevationInMeters': 980.0,
 'verbatimLatitude': None,
 'verbatimLongitude': None,
 'verbatimCoordinateSystem': 

In [102]:
client.close()